#📌 Extracción

In [1]:
import pandas as pd
import plotly.express as px

df = pd.read_json("https://raw.githubusercontent.com/sthemonica/alura-voz/refs/heads/main/Dados/Telco-Customer-Churn.json")

#🔧 Transformación

In [2]:
customer=pd.json_normalize(df["customer"])
customer['SeniorCitizen']=customer['SeniorCitizen'].replace({0:'No',1:'Yes'})
phone=pd.json_normalize(df["phone"])
internet=pd.json_normalize(df["internet"])
account=pd.json_normalize(df["account"])
df=pd.concat([df["customerID"],df["Churn"],customer,phone,internet,account],axis=1)
table={"Yes":"Si","Female":"Mujer","Male":"Hombre","No phone service":"Sin servicio de teléfono","No internet service":"Sin servicio de internet","One year":"Anual","Two year":"Bianual","Month-to-month":"Mensual","Fiber optic":"Fibra óptica",
       "Mailed check":"Cheque","Electronic check":"Cheque electrónico","Credit card (automatic)":"Tarjeta de crédito (automatico)","Bank transfer (automatic)":"Transferencia bancaria (automatico)"}
df=df.replace(table)

for col in df.columns:
  print(f"Valores únicos en la columna'{col}': {df[col].nunique()}")
  if df[col].nunique() < 50:
    print(df[col].unique())
  print('\n')

df = df[df['Churn'].str.strip() != '']

count = df.groupby(['Churn', 'tenure']).size().count()
print(count)
print(df)


Valores únicos en la columna'customerID': 7267


Valores únicos en la columna'Churn': 3
['No' 'Si' '']


Valores únicos en la columna'gender': 2
['Mujer' 'Hombre']


Valores únicos en la columna'SeniorCitizen': 2
['No' 'Si']


Valores únicos en la columna'Partner': 2
['Si' 'No']


Valores únicos en la columna'Dependents': 2
['Si' 'No']


Valores únicos en la columna'tenure': 73


Valores únicos en la columna'PhoneService': 2
['Si' 'No']


Valores únicos en la columna'MultipleLines': 3
['No' 'Si' 'Sin servicio de teléfono']


Valores únicos en la columna'InternetService': 3
['DSL' 'Fibra óptica' 'No']


Valores únicos en la columna'OnlineSecurity': 3
['No' 'Si' 'Sin servicio de internet']


Valores únicos en la columna'OnlineBackup': 3
['Si' 'No' 'Sin servicio de internet']


Valores únicos en la columna'DeviceProtection': 3
['No' 'Si' 'Sin servicio de internet']


Valores únicos en la columna'TechSupport': 3
['Si' 'No' 'Sin servicio de internet']


Valores únicos en la columna'Streamin

#📊 Carga y análisis

In [3]:
px.histogram(df, x = 'Churn', text_auto = True, color = 'Churn', barmode = 'group')

In [4]:
px.histogram(df, x = 'gender', text_auto = True, color = 'Churn', barmode = 'group',labels={'gender': 'Genero'})

In [5]:
px.histogram(df, x = 'SeniorCitizen', text_auto = True, color = 'Churn', barmode = 'group',labels={'SeniorCitizen': 'Tercera Edad'})

In [6]:
px.histogram(df, x = "Contract", text_auto = True, color = "Churn", barmode = "group",category_orders={"Contract": ["Mensual","Anual","Bianual"]},labels={"Contract":"Tipo de Contrato"})

In [7]:
px.histogram(df, x = 'PhoneService', text_auto = True, color = 'Churn', barmode = 'group',labels={'PhoneService':'Servicio de Telefono'})

In [8]:
px.histogram(df, x = 'InternetService', text_auto = True, color = 'Churn', barmode = 'group',labels={'InternetService':'Servicio de Internet'})

In [9]:
px.histogram(df, x = 'StreamingTV',  text_auto = True, color = 'Churn', barmode = 'group',labels={'StreamingTV':'Servicio de Streaming TV'})

In [10]:
px.histogram(df, x = 'PaymentMethod',  text_auto = True, color = 'Churn', barmode = 'group',labels={'PaymentMethod':'Medio de Pago'})

In [11]:
px.histogram(df, x = 'PaperlessBilling',  text_auto = True, color = 'Churn', barmode = 'group',labels={'PaperlessBilling':'Boleta Electronica'})

In [12]:
px.box(df, x = 'Churn', y ='Charges.Monthly', color = 'Churn',labels={'Charges.Monthly':'Cargos Mensuales'})

In [14]:
tenuream=df.groupby(['tenure','Churn']).size().reset_index(name='count')
tenuream["Si"]=tenuream.get("Yes",0)
tenuream["No"]=tenuream.get("No",0)
for index, row in tenuream.iterrows():
  if row["Churn"]=="Si":
    tenuream.at[index,"Si"]=row["count"]
  else:
    tenuream.at[index,"No"]=row["count"]
tenuream=tenuream.drop(columns=["Churn","count"])
tenuream=tenuream.groupby("tenure", as_index = False).aggregate({"Si": "sum", "No": "sum"}).reindex(columns=["tenure","Si","No"])
px.line(tenuream.reset_index(), x='tenure', y=['No','Si'], markers=True,labels={'tenure':'Permanencia'})

#📄Informe final

En este informe analizamos los datos referentes a las cancelaciones de clientes de la compañía Telecom X.

Tras la importación, normalización y análisis de los datos proporcionados, podemos sacar las siguientes conclusiones.

1- Los clientes con contratos mensuales tienen mayor probabilidad de cancelar sus contratos que los clientes con contratos anuales o bianuales.
Se recomienda incentivar a clientes a tomar contratos de mayor duración con promociones o descuentos temporales.

2- Mientras menor sea la antigüedad del cliente, mayor es la probabilidad de que cancele el contrato. Al igual que en el punto anterior, se recomienda sugerir contratos de periodos más largos.


3- Clientes que cancelan con cheque electrónico tienen una probabilidad mucho mayor de cancelar su contrato que clientes que pagan por otros medios.
Se recomienda tanto sugerir medios de pago distintos como ademas indagar cuál es el problema de quienes pagan con cheque electrónico, si es de parte de la empresa o del banco emisor.

4- Igualmente, clientes que reciben boleta electrónica tienen una probabilidad mucho mayor de cancelar su contrato que clientes que pagan por otros medios.
Se recomienda investigar por qué motivo ocurre, mientras que sugerir a clientes que puedan tener problemas de recibir esta, que reciban boleta física.

5- Por último, mientras mayores sean los cargos mensuales del cliente, mayor es la posibilidad de que cancele el contrato.Se recomienda empezar los contratos con valores menores para, después del periodo de mayor cancelación, poder aumentarlos paulatinamente.